This is the Employee Management System code, which has user authentication functionalities, CRUD operations on employee records, and performance review management.

In [ ]:
!pip install flask flask_sqlalchemy flask_bcrypt

from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///employees.db'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(200), nullable=False)

class Employee(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    role = db.Column(db.String(80), nullable=False)
    salary = db.Column(db.Float, nullable=False)
    performance_review = db.Column(db.String(200), nullable=True)


db.create_all()

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password']).decode('utf-8')
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message': 'User registered successfully'})

# User login endpoint
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()
    if user and bcrypt.check_password_hash(user.password, data['password']):
        return jsonify({'message': 'Login successful'})
    return jsonify({'message': 'Invalid credentials'}), 401

@app.route('/employee', methods=['POST'])
def add_employee():
    data = request.get_json()
    new_employee = Employee(name=data['name'], role=data['role'], salary=data['salary'])
    db.session.add(new_employee)
    db.session.commit()
    return jsonify({'message': 'Employee added successfully'})

@app.route('/employees', methods=['GET'])
def get_employees():
    employees = Employee.query.all()
    output = []
    for employee in employees:
        employee_data = {
            'id': employee.id,
            'name': employee.name,
            'role': employee.role,
            'salary': employee.salary,
            'performance_review': employee.performance_review
        }
        output.append(employee_data)
    return jsonify({'employees': output})


@app.route('/employee/<int:id>', methods=['GET'])
def get_employee(id):
    employee = Employee.query.get(id)
    if not employee:
        return jsonify({'message': 'Employee not found'}), 404
    employee_data = {
        'id': employee.id,
        'name': employee.name,
        'role': employee.role,
        'salary': employee.salary,
        'performance_review': employee.performance_review
    }
    return jsonify(employee_data)


@app.route('/employee/<int:id>', methods=['PUT'])
def update_employee(id):
    data = request.get_json()
    employee = Employee.query.get(id)
    if not employee:
        return jsonify({'message': 'Employee not found'}), 404
    employee.name = data['name']
    employee.role = data['role']
    employee.salary = data['salary']
    db.session.commit()
    return jsonify({'message': 'Employee details updated successfully'})

@app.route('/employee/<int:id>', methods=['DELETE'])
def delete_employee(id):
    employee = Employee.query.get(id)
    if not employee:
        return jsonify({'message': 'Employee not found'}), 404
    db.session.delete(employee)
    db.session.commit()
    return jsonify({'message': 'Employee deleted successfully'})

@app.route('/employee/<int:id>/review', methods=['PUT'])
def add_update_review(id):
    data = request.get_json()
    employee = Employee.query.get(id)
    if not employee:
        return jsonify({'message': 'Employee not found'}), 404
    employee.performance_review = data['review']
    db.session.commit()
    return jsonify({'message': 'Performance review updated successfully'})


if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(port='5000')
print(f"Public URL: {public_url}")
app.run()